In this notebook, we will try to get logistic regression running.

Helpful documentation: https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html 

In [1]:
import pandas as pd

In [2]:
# load data
data_path = "cleaned_data.csv" # get the data without the disagreements
data = pd.read_csv(data_path)[['text','sentiment']]
print(data.shape)

(38961, 2)


In [3]:
data

,text,sentiment
0,Spent the entire morning in a meeting w/ a ven...,neutral
1,Oh! Good idea about putting them on ice cream,positive
2,says good (or should i say bad?) afternoon! h...,neutral
3,haha better drunken tweeting you mean?,positive
4,had an awsome salad! I recommend getting the S...,positive
...,...,...
38956,RT @toricolelli: My phones been charging for a...,negative
38957,'@WhoaBiebz: GET YOUR SHIT TOGETHER OR I'LL GU...,negative
38958,Those** PICK UP THE SLACK YOU FUCK BOYS @Apple,negative
38959,@umo_games @Apple ended up getting a new compu...,neutral


In [1]:
# how to extract features? TfIdf, countvectorizer, bow, etc.